In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [4]:
def scrape_wiki(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
    else:
        print("Error fetching the webpage. Status code:", response.status_code)
    table_data = []
    first_tag = soup.find('h3')
    state = first_tag.text
    state = state.replace('[edit]','')
 
    table = soup.find('table',class_='wikitable')  
    table_list = load_table(table, 'state', state)
    poll_count = 0
    for row in table_list:
        if poll_count <= 15:
            table_data.append(row)
            poll_count += 1

    
    table_header = ['Poll Source', 'Date administered (2004)', 'John Kerry', 'George W. Bush', 'Margin', 'Sample size', 'Margin of error', 'state']
    
    headers = first_tag.find_all_next('h3')
    for header in headers:
        state = header.text
        state = state.replace('[edit]','') 
        next_table = header.find_next_sibling('table', class_='wikitable')
        if next_table:
            t_date  = load_table(next_table, 'state', state)
            poll_count = 0
            for t in t_date:
                if t != table_header:
                    if poll_count <= 15: 
                        table_data.append(t)
                        poll_count += 1
                    
    return table_data




        
def load_table(table, descriptor_title = 'none', descriptor = 'none'):
    t = table
    table_data = []
    row_data = []
    rows = t.find_all('tr')
    heads = t.find_all('th')
    heads = [head.text.strip() for head in heads]
    if descriptor_title != 'none':
        heads.append(descriptor_title)
    
    table_data.append(heads)

    # print(heads)

    for row in rows:
        cols = row.find_all('td')
        cols = [col.text.strip() for col in cols]
        if descriptor != 'none':
            cols.append(descriptor)
        if len(cols) > 1:
            table_data.append(cols)         
    return table_data


In [5]:
url = 'https://en.wikipedia.org/wiki/Statewide_opinion_polling_for_the_2004_United_States_presidential_election'
polls = scrape_wiki(url)

In [9]:
polling = pd.DataFrame(polls)
polling.columns = polling.iloc[0]
polling = polling[1:]

In [10]:
polling.head()

,Poll Source,Date administered (2004),John Kerry,George W. Bush,Margin,Sample size,Margin of error,state,None
1,Rasmussen Reports/Pulse Opinion Research,October 21–30,42%,53%,11,Not reported,±5%,Alabama,None
2,WKRG-TV Mobile/SurveyUSA,October 25–27,39%,57%,18,634 LV,±4%,Alabama,None
3,Rasmussen Reports/Pulse Opinion Research,October 6–20,41%,53%,12,Not reported,±5%,Alabama,None
4,WKRG-TV Mobile/SurveyUSA,October 1–3,34%,62%,28,600 LV,±4%,Alabama,None
5,Rasmussen Reports/Pulse Opinion Research,September 15–28,40%,56%,16,Not reported,±5%,Alabama,None


In [13]:
polling.loc[polling.state == 'Florida']

,Poll Source,Date administered (2004),John Kerry,George W. Bush,Margin,Sample size,Margin of error,state,None
98,Zogby International (Daily Tracking),October 29–November 1,48%,48%,Tied,601 LV,±4.1%,Florida,None
99,WFLA-TV Tampa/WPTV West Palm/WKMG-TV Orlando/W...,October 29–31,48%,49%,1,742 LV,±3.7%,Florida,None
100,MSNBC/Knight-Ridder/Mason-Dixon Polling & Rese...,October 27–29,45%,49%,4,625 LV,±4%,Florida,None
101,Zogby International (Daily Tracking),October 25–28,47%,48%,1,601 LV,±4.1%,Florida,None
102,Rasmussen Reports/Pulse Opinion Research,October 27,45%,50%,5,500 LV,±4%,Florida,None
103,America Coming Together/Hamilton Beattie (D),October 25–27,46%,48%,2,Not reported,Not reported,Florida,None
104,Rasmussen Reports/Pulse Opinion Research (Dail...,October 20–26,48%,48%,Tied,567 LV,±4%,Florida,None
105,American Research Group,October 23–25,49%,47%,2,600 LV,±4%,Florida,None
106,WFLA-TV Tampa/WPTV West Palm/WKMG-TV Orlando/W...,October 22–24,50%,48%,2,749 LV,±3.7%,Florida,None
107,Zogby International (Daily Tracking),October 21–24,46%,49%,3,602 LV,±4.1%,Florida,None


In [14]:
polling.to_csv('C:/Users/appar/OneDrive/pres_election_model_2024/polls_2004.csv')